### 示例
#### 1. 找到所有频繁项集
#### 
##### 第一步生成初始候选集，一般是所有订单里面出现的不重复所有项目
##### 第二步对初始候选集，筛选频繁项集 （支持度大于某阈值的留下）
##### 第三步将初始候选集形成的频繁项集生成第一个候选k项集（第一个频繁项集中的元素两两组合）
##### 第四步将第一个候选k项集再次形成一个频繁项集，此频繁项集再形成第二个候选k项集（第二个频繁项集中的元素两两组合），第二个候选k项集再形成频繁项集，以此类推一直循环下去，直到找不到频繁项集为止
##### 前四步形成了所有的频繁项集
##### 
#### 2. 生成关联规则
#### 
##### 第一步 循环所有的频繁项集，每一条频繁项集形成此频繁项集能够形成的所有子集
##### 
#### 3. 计算条件概率
#### 
##### 第二步 计算所有候选项集的支持度（包含所有频繁项集及其子集，非频繁项集的支持度信息），用此条频繁项集的支持度/此条频繁项集形成的所有子集的支持度->计算出条件概率 （在某子集出现的情况下此条频繁项集的条件概率是多少）
##### 第三步 找到条件概率高于某阈值的组合

In [162]:
dataset=[['苹果','香蕉','橘子'],['榴莲','香蕉','哈密瓜'],['苹果','香蕉','榴莲','哈密瓜'],['橘子','榴莲']]

In [42]:
dataset

[['苹果', '香蕉', '橘子'],
 ['榴莲', '香蕉', '哈密瓜'],
 ['苹果', '香蕉', '榴莲', '哈密瓜'],
 ['橘子', '榴莲']]

#### 生成候选项集

In [25]:
def create_cl(dataset):
    cl=[]
    for transaction in dataset:
        for item in transaction:
            if not {item} in cl:
                cl.append({item})
    cl.sort()
    return list(map(frozenset,cl))

In [26]:
create_cl(dataset)

[frozenset({'苹果'}),
 frozenset({'香蕉'}),
 frozenset({'橘子'}),
 frozenset({'榴莲'}),
 frozenset({'哈密瓜'})]

#### 生成频繁项集

In [38]:
def create_freq_transaction(dataset,ck,min_support=0.5):
    sscnt={}
    for transaction in dataset:
        for can in ck:
            if  can.issubset(transaction):
                sscnt[can]=sscnt.get(can,0)+1
    num_transactions=float(len(dataset))
    freq_transaction=[]
    support_data={}
    for key in sscnt:
        support=sscnt[key]/num_transactions
        support_data[key]=support
        if support>=min_support:
            freq_transaction.append(key)
    return support_data,freq_transaction
                
            

In [39]:
cl=create_cl(dataset)
support_data,freq_transaction=create_freq_transaction(dataset,cl,min_support=0.5)

In [40]:
support_data

{frozenset({'苹果'}): 0.5,
 frozenset({'香蕉'}): 0.75,
 frozenset({'橘子'}): 0.5,
 frozenset({'榴莲'}): 0.75,
 frozenset({'哈密瓜'}): 0.5}

In [41]:
freq_transaction

[frozenset({'苹果'}),
 frozenset({'香蕉'}),
 frozenset({'橘子'}),
 frozenset({'榴莲'}),
 frozenset({'哈密瓜'})]

#### 生成第Ck个候选集

In [54]:
def create_ck(freq_k_transaction):
    ck=[]
    k=len(freq_k_transaction)
    for i in range(k):
        for j in range(i+1,k):
            t1=freq_k_transaction[i]
            t2=freq_k_transaction[j]
            t=t1|t2
            if(not t in ck)and(len(t)==len(freq_k_transaction[0])+1):
                ck.append(t)
    return ck             
    

In [55]:
create_ck(freq_transaction)

[frozenset({'苹果', '香蕉'}),
 frozenset({'橘子', '苹果'}),
 frozenset({'榴莲', '苹果'}),
 frozenset({'哈密瓜', '苹果'}),
 frozenset({'橘子', '香蕉'}),
 frozenset({'榴莲', '香蕉'}),
 frozenset({'哈密瓜', '香蕉'}),
 frozenset({'榴莲', '橘子'}),
 frozenset({'哈密瓜', '橘子'}),
 frozenset({'哈密瓜', '榴莲'})]

In [51]:
zz=frozenset({'苹果'})|frozenset({'哈密瓜'})

In [53]:
zz.issubset(['苹果', '香蕉', '榴莲', '哈密瓜'])

True

#### Apriori函数

In [62]:
def Apriori(dataset,min_support=0.5):
    cl=create_cl(dataset)
    support_data_1,freq_transaction_1=create_freq_transaction(dataset,cl,min_support=min_support)
    all_freq_transaction=[freq_transaction_1]
    while len(all_freq_transaction[-1])>0:
        
        ck=create_ck(all_freq_transaction[-1])
        support_data_k,freq_transaction_k=create_freq_transaction(dataset,ck,min_support=min_support)
        support_data_1.update(support_data_k)
        all_freq_transaction.append(freq_transaction_k)
    return support_data_1,all_freq_transaction    
    
    

In [163]:
s,fs=Apriori(dataset,0.5)

In [164]:
s

{frozenset({'苹果'}): 0.5,
 frozenset({'香蕉'}): 0.75,
 frozenset({'橘子'}): 0.5,
 frozenset({'榴莲'}): 0.75,
 frozenset({'哈密瓜'}): 0.5,
 frozenset({'苹果', '香蕉'}): 0.5,
 frozenset({'橘子', '苹果'}): 0.25,
 frozenset({'橘子', '香蕉'}): 0.25,
 frozenset({'榴莲', '香蕉'}): 0.5,
 frozenset({'哈密瓜', '香蕉'}): 0.5,
 frozenset({'哈密瓜', '榴莲'}): 0.5,
 frozenset({'榴莲', '苹果'}): 0.25,
 frozenset({'哈密瓜', '苹果'}): 0.25,
 frozenset({'榴莲', '橘子'}): 0.25,
 frozenset({'哈密瓜', '榴莲', '香蕉'}): 0.5,
 frozenset({'榴莲', '苹果', '香蕉'}): 0.25,
 frozenset({'哈密瓜', '苹果', '香蕉'}): 0.25}

In [165]:
fs

[[frozenset({'苹果'}),
  frozenset({'香蕉'}),
  frozenset({'橘子'}),
  frozenset({'榴莲'}),
  frozenset({'哈密瓜'})],
 [frozenset({'苹果', '香蕉'}),
  frozenset({'榴莲', '香蕉'}),
  frozenset({'哈密瓜', '香蕉'}),
  frozenset({'哈密瓜', '榴莲'})],
 [frozenset({'哈密瓜', '榴莲', '香蕉'})],
 []]

#### 产生关联规则

#### 生成全部子集

In [123]:
def create_subset(fromlist,tolist):
    for i in range(len(fromlist)):
        t=[fromlist[i]]
        tt=frozenset(set(fromlist)-set(t))
        if not tt in tolist:
            tolist.append(tt)
            tt=list(tt)
            if len(tt)>1:
                create_subset(tt,tolist)
    return None

In [114]:
fromlist= ['a','b','c','d']
tolist=[]
create_subset(fromlist,tolist)
tolist

[frozenset({'b', 'c', 'd'}),
 frozenset({'b', 'd'}),
 frozenset({'b'}),
 frozenset({'d'}),
 frozenset({'b', 'c'}),
 frozenset({'c'}),
 frozenset({'c', 'd'}),
 frozenset({'a', 'c', 'd'}),
 frozenset({'a', 'd'}),
 frozenset({'a'}),
 frozenset({'a', 'c'}),
 frozenset({'a', 'b', 'd'}),
 frozenset({'a', 'b'}),
 frozenset({'a', 'b', 'c'})]

In [147]:
fromlist= ['a','b']
tolist=[]
create_subset(fromlist,tolist)
tolist

[frozenset({'b'}), frozenset({'a'})]

In [80]:
len(list(['ab']))

1

In [92]:
set([frozenset({'苹果', '香蕉'}),frozenset({'榴莲', '香蕉'}),frozenset({'哈密瓜', '香蕉'}),frozenset({'哈密瓜', '榴莲'})])

{frozenset({'哈密瓜', '榴莲'}),
 frozenset({'哈密瓜', '香蕉'}),
 frozenset({'榴莲', '香蕉'}),
 frozenset({'苹果', '香蕉'})}

In [94]:
len(frozenset({'哈密瓜', '榴莲'}))

2

In [95]:
frozenset({'哈密瓜'}) in frozenset({'哈密瓜', '榴莲'})

False

In [100]:
frozenset(set(frozenset({'哈密瓜', '榴莲'}))-set(frozenset({'哈密瓜'})))

frozenset({'榴莲'})

In [2]:
frozenset(set(frozenset({'哈密瓜'}))-set(frozenset({'哈密瓜'})))

frozenset()

In [108]:
set(frozenset({'哈密瓜', '榴莲'}))

{'哈密瓜', '榴莲'}

In [109]:
set(frozenset({'哈密瓜', '榴莲','荔枝'}))

{'哈密瓜', '榴莲', '荔枝'}

#### 计算条件概率
###### P(A)
###### P(A|B)=P(AB)/P(B)
###### P(A)提升度=P(A|B)/P(A)=P(AB)/P(A)P(B)

In [143]:
def cal_conf(fre_set,h,support_data,rulelist,min_conf):
    for after in h:
        #after在fre_set-after的条件下的概率
        conf=support_data[fre_set]/support_data[fre_set-after]
        #after在fre_set-after的条件下的提升度
        lift=support_data[fre_set]/(support_data[fre_set-after]*support_data[after])
        if conf>=min_conf and lift>1:
            print(fre_set-after,'-->',after,'\n',
                 'before support:',support_data[fre_set-after],',',
                  'after support:',support_data[after],',',
                  'together support:',support_data[fre_set],',',
                  'conf:',conf,',',
                  'list:',lift)
            rulelist.append((fre_set-after,after,conf))
    return None


In [116]:
set([1,2])|set([3])

{1, 2, 3}

In [117]:
frozenset({'哈密瓜', '榴莲'})-frozenset({'哈密瓜'})

frozenset({'榴莲'})

In [168]:
h=[]
fromlis=list(frozenset({'哈密瓜', '榴莲', '香蕉'}))
create_subset(fromlis,h)
h

[frozenset({'哈密瓜', '榴莲'}),
 frozenset({'哈密瓜'}),
 frozenset({'榴莲'}),
 frozenset({'哈密瓜', '香蕉'}),
 frozenset({'香蕉'}),
 frozenset({'榴莲', '香蕉'})]

In [122]:
list(frozenset({'哈密瓜', '榴莲', '香蕉'}))

['香蕉', '榴莲', '哈密瓜']

In [169]:
rulelist=[]
cal_conf(frozenset({'哈密瓜', '榴莲', '香蕉'}),h,s,rulelist,min_conf=0.5)
rulelist

frozenset({'香蕉'}) --> frozenset({'榴莲', '哈密瓜'}) 
 before support: 0.75 , after support: 0.5 , together support: 0.5 , conf: 0.6666666666666666 , list: 1.3333333333333333
frozenset({'香蕉', '榴莲'}) --> frozenset({'哈密瓜'}) 
 before support: 0.5 , after support: 0.5 , together support: 0.5 , conf: 1.0 , list: 2.0
frozenset({'香蕉', '哈密瓜'}) --> frozenset({'榴莲'}) 
 before support: 0.5 , after support: 0.75 , together support: 0.5 , conf: 1.0 , list: 1.3333333333333333
frozenset({'榴莲'}) --> frozenset({'香蕉', '哈密瓜'}) 
 before support: 0.75 , after support: 0.5 , together support: 0.5 , conf: 0.6666666666666666 , list: 1.3333333333333333
frozenset({'哈密瓜', '榴莲'}) --> frozenset({'香蕉'}) 
 before support: 0.5 , after support: 0.75 , together support: 0.5 , conf: 1.0 , list: 1.3333333333333333
frozenset({'哈密瓜'}) --> frozenset({'香蕉', '榴莲'}) 
 before support: 0.5 , after support: 0.5 , together support: 0.5 , conf: 1.0 , list: 2.0


[(frozenset({'香蕉'}), frozenset({'哈密瓜', '榴莲'}), 0.6666666666666666),
 (frozenset({'榴莲', '香蕉'}), frozenset({'哈密瓜'}), 1.0),
 (frozenset({'哈密瓜', '香蕉'}), frozenset({'榴莲'}), 1.0),
 (frozenset({'榴莲'}), frozenset({'哈密瓜', '香蕉'}), 0.6666666666666666),
 (frozenset({'哈密瓜', '榴莲'}), frozenset({'香蕉'}), 1.0),
 (frozenset({'哈密瓜'}), frozenset({'榴莲', '香蕉'}), 1.0)]

In [167]:
s

{frozenset({'苹果'}): 0.5,
 frozenset({'香蕉'}): 0.75,
 frozenset({'橘子'}): 0.5,
 frozenset({'榴莲'}): 0.75,
 frozenset({'哈密瓜'}): 0.5,
 frozenset({'苹果', '香蕉'}): 0.5,
 frozenset({'橘子', '苹果'}): 0.25,
 frozenset({'橘子', '香蕉'}): 0.25,
 frozenset({'榴莲', '香蕉'}): 0.5,
 frozenset({'哈密瓜', '香蕉'}): 0.5,
 frozenset({'哈密瓜', '榴莲'}): 0.5,
 frozenset({'榴莲', '苹果'}): 0.25,
 frozenset({'哈密瓜', '苹果'}): 0.25,
 frozenset({'榴莲', '橘子'}): 0.25,
 frozenset({'哈密瓜', '榴莲', '香蕉'}): 0.5,
 frozenset({'榴莲', '苹果', '香蕉'}): 0.25,
 frozenset({'哈密瓜', '苹果', '香蕉'}): 0.25}

#### 产生关联规则主函数

In [149]:
def create_rules(support_data,all_freq_transaction,min_conf=0.8):
    all_rulelist=[]
    for i in range(1,len(all_freq_transaction)):
        for fre_set in all_freq_transaction[i]:
            fre_list=list(fre_set)
            all_subset=[]
            create_subset(fre_list,all_subset)
            cal_conf(fre_set,all_subset,support_data,all_rulelist,min_conf)
    return all_rulelist        

In [151]:
create_rules(s,fs,0.6)

frozenset({'苹果'}) --> frozenset({'香蕉'}) 
 before support: 0.5 , after support: 0.75 , together support: 0.5 , conf: 1.0 , list: 1.3333333333333333
frozenset({'香蕉'}) --> frozenset({'苹果'}) 
 before support: 0.75 , after support: 0.5 , together support: 0.5 , conf: 0.6666666666666666 , list: 1.3333333333333333
frozenset({'香蕉'}) --> frozenset({'哈密瓜'}) 
 before support: 0.75 , after support: 0.5 , together support: 0.5 , conf: 0.6666666666666666 , list: 1.3333333333333333
frozenset({'哈密瓜'}) --> frozenset({'香蕉'}) 
 before support: 0.5 , after support: 0.75 , together support: 0.5 , conf: 1.0 , list: 1.3333333333333333
frozenset({'哈密瓜'}) --> frozenset({'榴莲'}) 
 before support: 0.5 , after support: 0.75 , together support: 0.5 , conf: 1.0 , list: 1.3333333333333333
frozenset({'榴莲'}) --> frozenset({'哈密瓜'}) 
 before support: 0.75 , after support: 0.5 , together support: 0.5 , conf: 0.6666666666666666 , list: 1.3333333333333333
frozenset({'香蕉'}) --> frozenset({'哈密瓜', '榴莲'}) 
 before support: 0.75

[(frozenset({'苹果'}), frozenset({'香蕉'}), 1.0),
 (frozenset({'香蕉'}), frozenset({'苹果'}), 0.6666666666666666),
 (frozenset({'香蕉'}), frozenset({'哈密瓜'}), 0.6666666666666666),
 (frozenset({'哈密瓜'}), frozenset({'香蕉'}), 1.0),
 (frozenset({'哈密瓜'}), frozenset({'榴莲'}), 1.0),
 (frozenset({'榴莲'}), frozenset({'哈密瓜'}), 0.6666666666666666),
 (frozenset({'香蕉'}), frozenset({'哈密瓜', '榴莲'}), 0.6666666666666666),
 (frozenset({'哈密瓜', '香蕉'}), frozenset({'榴莲'}), 1.0),
 (frozenset({'榴莲', '香蕉'}), frozenset({'哈密瓜'}), 1.0),
 (frozenset({'哈密瓜'}), frozenset({'榴莲', '香蕉'}), 1.0),
 (frozenset({'哈密瓜', '榴莲'}), frozenset({'香蕉'}), 1.0),
 (frozenset({'榴莲'}), frozenset({'哈密瓜', '香蕉'}), 0.6666666666666666)]

In [172]:
import pandas as pd
df=pd.DataFrame({'A':[1,2,3,4],'B':[6,7,8,9]},index=[1,1,2,2])
df

,A,B
1,1,6
1,2,7
2,3,8
2,4,9


In [176]:
list(df.loc[2,'A'])

[3, 4]

In [8]:
frozenset({'苹果', '香蕉'})-frozenset({'苹果'})

frozenset({'香蕉'})